In [1]:
from multiprocessing import Pool
from euler import is_prime, n_digits
import sys
from functools import partial, reduce
from itertools import combinations, permutations
from tqdm import tqdm_notebook as tqdm

In [2]:
# def is_prime(n):
#     if n == 2 or n == 3:
#         return True
#     if n == 1 or n%2 == 0: 
#         return False
#     for i in range(
#         3, 
#         int(n**0.5) + 1,
#         2
#     ):
#         if n%i == 0:
#             return False    
#     return True

# def n_digits(n):
#     return len(str(n))

In [3]:
def primes_up_to(n):
    if n == 3:
        return [2, 3]
    primes = primes_up_to(n-1)
    for i in [p for p in primes if p <= int(n**0.5)]:
        if n%i == 0:
            return primes
    primes.append(n)
    return primes

In [4]:
print(primes_up_to(100))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [5]:
print([n for n in range(1, 100) if is_prime(n)])

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [6]:
print([n_digits(n) for n in range(1, 100) if is_prime(n)])

[1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [7]:
pool = Pool(6)

In [8]:
%%time

primes = [
    n + 1 
    for n, is_p 
    in enumerate(pool.map(
        is_prime, 
        range(1, int(1e6))
    )) 
    if is_p
]
print(len(primes))

78498
Wall time: 595 ms


In [9]:
%%time

i = 0
for p in primes:
    if is_prime(673*10**n_digits(p) + p) and is_prime(p*1000 + 673):
        print(p)
        i += 1
        if i == 5:
            break

3
7
109
199
397
Wall time: 6.95 ms


In [10]:
# def is_prime(n, primes_smaller):
#     for i in primes_smaller:
#         if n%i == 0:
#             return False    
#     return True

In [11]:
print([n for n in range(10, 100) if is_prime(n, [2, 3, 5, 7])])

[11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [12]:
%%time

primes = [2, 3, 5, 7]
for i in range(5):
    range_min = 2**(2**i)
    new_primes = [
        n + range_min 
        for n, is_p 
        in enumerate(pool.map(
            partial(is_prime, primes_smaller=primes), 
            range(range_min, min(range_min**2, int(1e6)))
        )) 
        if is_p
    ]
    primes += new_primes
print(len(primes))

78498
Wall time: 6.75 s


In [13]:
range_min**2 / int(1e6)

4294.967296

In [14]:
%%time

i = 0
for p in tqdm(primes):
    if all(pool.map(
        is_prime, 
        [
#             673*10**n_digits(p) + p,
#             p*1000 + 673,
            109*10**n_digits(p) + p,
            p*1000 + 109,
            7*10**n_digits(p) + p,
            p*10 + 7,
            3*10**n_digits(p) + p,
            p*10 + 3
        ])):
        print(p)
        i += 1
        if i == 3:
            break

673
29059
79693
Wall time: 2.32 s


In [15]:
print(
    list(combinations([1, 2, 3], 1)), 
    list(combinations([1, 2, 3], 2)), 
    list(combinations([1, 2, 3], 3)),
    sep='\n'
)

[(1,), (2,), (3,)]
[(1, 2), (1, 3), (2, 3)]
[(1, 2, 3)]


In [16]:
123
124, 134, 234
125, 135, 235, 145, 245, 345

(125, 135, 235, 145, 245, 345)

In [17]:
list(combinations([1, 2, 3, 4, 5], 3))

[(1, 2, 3),
 (1, 2, 4),
 (1, 2, 5),
 (1, 3, 4),
 (1, 3, 5),
 (1, 4, 5),
 (2, 3, 4),
 (2, 3, 5),
 (2, 4, 5),
 (3, 4, 5)]

In [18]:
[x + (5, ) for x in combinations([1, 2, 3, 4], 2)]

[(1, 2, 5), (1, 3, 5), (1, 4, 5), (2, 3, 5), (2, 4, 5), (3, 4, 5)]

In [19]:
[x + (11, ) for x in combinations(primes[1:4], 2)]

[(3, 5, 11), (3, 7, 11), (5, 7, 11)]

In [20]:
[x*10**n_digits(y) + y for x, y in permutations((7, 23, 109), 2)]

[723, 7109, 237, 23109, 1097, 10923]

In [21]:
primes[128]

727

In [22]:
%%time

n_tuples_found = 0
tuple_size = 4
for i, p in tqdm(enumerate(primes[:32]), total=32):
    if i < tuple_size:
        continue
    p_combs = [
        comb + (p, ) 
        for comb 
        in combinations(
            primes[1:i], # skip 2
            tuple_size - 1
        )
    ]
    for p_comb in p_combs:
        if all(pool.map(
            is_prime, 
            [
                p1*10**n_digits(p2) + p2 
                for p1, p2 
                in permutations(p_comb, 2) # 2 means we concat 2 primes and check if the concat is prime
            ]
        )):
            print(p_comb)
            n_tuples_found += 1
    if n_tuples_found == 1:
        break


Wall time: 14.6 s


In [26]:
%%time

# n_tuples_found = 0
primes_in = primes[1:128] # skip 2
for tuple_size in range(2, 5):
    primes_out = set()
    for i, p in tqdm(enumerate(primes_in), total=len(primes_in)):
        if i < tuple_size:
            continue
        p_combs = [
            comb + (p, ) 
            for comb 
            in combinations(
                primes_in[:i],
                tuple_size - 1
            )
        ]
        for p_comb in p_combs:
            if all(pool.map(
                partial(is_prime, primes_smaller=primes_in), 
                [
                    p1*10**n_digits(p2) + p2 
                    for p1, p2 
                    in permutations(p_comb, 2) # 2 means we concat 2 primes and check if the concat is prime
                ]
            )):
                primes_out.update(p_comb)
    #             print(p, p_comb)
    #             print(primes_out)
    #             n_tuples_found += 1
    #     if n_tuples_found == 1:
    #         break
    primes_out = sorted(primes_out)
    print(f'Tuples of size {tuple_size} is processed. \nThere are {len(primes_out)} primes left out of {len(primes_in)}')
    primes_in = primes_out


Tuples of size 2 is processed. 
There are 126 primes left out of 127



Tuples of size 3 is processed. 
There are 98 primes left out of 126



Tuples of size 4 is processed. 
There are 38 primes left out of 98
Wall time: 36min 15s


In [24]:
len(primes_out)

4

In [25]:
print(primes_out)

[3, 7, 109, 673]
